# Re-implementation of AudioLM

In the following notebook, we are going to introduce a reimplementation of the **AudioLM** network, proposed in the paper *"AudioLM: a Language Modeling Approach to Audio Generation"* (https://arxiv.org/abs/2209.03143).

AudioLM is a state-of-the-art framework built in order to **generate high-quality audio**, while dealing with **long-term consistency**. Trained on a large corpora of audio data, AudioLM is able to provide **natural and coherent audio continuations**, given short initial prompts. The network is also able to **maintain speaker identity**, finding a good trade-off between audio quality and semantical coherence. AudioLM is also able to provide good quality musical continuation from a short prompt, but this will not be discussed in this notebook.


## Import and settings

In [28]:
# basic import
import torch
import numpy as np
import random
import pytorch_lightning as pl
import os

# data imports
from hubertKM import SemanticTokenizer
from SoundStream import soundstream_16khz
from data import TokensDataset, store_from_librilight
from torch.utils.data import DataLoader, random_split

# model imports
from TransformerModel import SemanticTransformer, CoarseTransformer, FineTransformer

In [6]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If using CUDA
np.random.seed(seed)
random.seed(seed)

## Converting Audio Data into Tokens

The most important novelty provided by AudioLM is the usage of a **mixed tokenization approach**, which has never been seen in other Language Modeling competitors. As shown below, we have two tokenization processes that can proceed in parallel.

<center><img src="reportImg/1.png"/></center>

In order to keep informations regarding language syntax and semantic content in speech, the audio waveform is passed through a **w2v-BERT** model that, combined with a K-Means quantizer, returns a set of **Semantic tokens**.

On the other hand, the network needs also to maintain informations about the acoustic features of the audio, in particular pronunciation and speaker identity. In order to do so, the audio waveform is passed through a pretrained audio codec, **SoundStream**, which is able to build an internal hierarchical representation of the audio. Through those representations, called **Acoustic tokens**, the audio is divided into different components, going from the most basic structural audio features (defined as **Coarse Acoustic tokens**) to the fine acoustic details (defined as **Fine Acoustic tokens**).

By modeling both semantic and acoustic tokens within the same framework, the semantic tokens would ensure long-term consistency, while the acoustic tokens would ensure high-quality audio synthesis.

<center><img src="reportImg/2.png"/></center>


### Semantic tokens with w2v-BERT-like model

In [8]:
w2vBERT = SemanticTokenizer("facebook/hubert-base-ls960","./hubertKM/hubert_base_ls960_L9_km500.bin") 

C:\Users\Gabriele\Desktop\NN_Project-AudioLM_2\env\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
## missing model Test

In [ ]:
## missing visualization

### Acoustic tokens with SoundStream codec model

In [10]:
soundStream = soundstream_16khz()

In [11]:
## missing model Test

### Token Dataset creation

In [13]:
tokenPath = "out" ## output file directory
tokenFile = "tokens.csv" ## output file name
audioPath = "data" ## data_location

In [14]:
#fileCount = storeTokens(audioPath, tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10)
fileCount = store_from_librilight(tokenPath, tokenFile, w2vBERT, soundStream, fileCountCheckpoint = 10, subset = "10h")

100%|███████████████████████████████████████████████████████████████████████████████| 570M/570M [00:38<00:00, 15.7MB/s]


SAVED 10 AUDIO ON OUTPUT out\tokens.csv. Total of 10 records saved.


KeyboardInterrupt: 

In [15]:
AUDIO_LENGTH = 30
CROP_LENGTH = [15,5,1]

semanticDataset = TokensDataset(tokenPath, tokenFile, mode = "semantic", expected_audio_length = AUDIO_LENGTH, crop_length = CROP_LENGTH)
coarseDataset = TokensDataset(tokenPath, tokenFile, mode = "coarse", expected_audio_length = AUDIO_LENGTH, crop_length = CROP_LENGTH)
fineDataset = TokensDataset(tokenPath, tokenFile, mode = "fine", expected_audio_length = AUDIO_LENGTH, crop_length = CROP_LENGTH)

## AudioLM: a transformer-based audio model

Once we have converted our data into token sequences, we can start defining the **generator** model. AudioLM network is based on three Decoder-only transformers, each of them dedicated to the **auto-regressive** generation of a specific kind of token. 
During inference, we first generate the new semantic tokens, and then use them to condition the generation of new acoustic tokens. With this structure, we can safely assume that *semantic tokens are expected to be conditionally independent from past acoustic tokens* given past semantic tokens:
$$
  p(z_{t}|z_{\lt t},y_{\lt t}) \simeq p(z_{t}|z_{\lt t})
$$



### Some implementation details

In the original paper, each generator follows **the same identical decoder-only Transformer** structure, with:
- 12 decoder layers
- 16 attention heads
- 1024 as embedding dimension
- 4096 as dimension for the feed-forward layer
- 0.1 as dropout value

The original model is also enriched with a **T5-style relative positional encoding**, that is trained along with the three stage transformers. For this reimplementation, we adapted an implementation of the paper *"Self-Attention with Relative Position Representations"* provided by https://github.com/AliHaiderAhmad001/Self-Attention-with-Relative-Position-Representations.

### Semantic Transformer: expanding a sentence snippet

<center><img src="reportImg/semantic.png"/></center>

The first transformer is responsible for the generation of new semantic tokens, estimating $ p(z_{t}|z_{\lt t}) $, where $ z_{i} $ denotes the $i$-th semantic token. In this way, given the semantic aspects of the prompt, the network is able to generate **new coherent content**, keeping long-term consistency.

In [26]:
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16

train_dataset, valid_dataset = random_split(coarseDataset, [TRAINING_PERCENTAGE, 1 - TRAINING_PERCENTAGE])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    log_every_n_steps=1,
    #devices=1 if torch.cuda.is_available() else None
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "checkpoints/semantic-checkpoint-10h-colab.ckpt"
d_model=256 #1024
num_layers=3 #12
num_heads=4 #16
dim_feedforward=1024 #4096
audioDuration=15 #30
vocab_size=500


if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming old model...")
    model = SemanticTransformer.load_from_checkpoint(checkpoint_path, d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)
else:
    print("No checkpoint found. Starting from scratch...")
    model = SemanticTransformer(d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)

No checkpoint found. Starting from scratch...


In [ ]:
model.train()
if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming training...")
    # Pass the checkpoint path to trainer.fit
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader, ckpt_path=checkpoint_path)
else:
    print("No checkpoint found. Starting from scratch...")
    # Start training from scratch
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

### Coarse Transformer: generating new audio

<center><img src="reportImg/coarse.png"/></center>

Once we have defined a mechanism to generate new semantic content, we need to generate the corresponding acoustic content. Since we want the new tokens to be **coherent both with the sentence meaning** (represented by the Semantic tokens) and the **original speaker and enviroment conditions** (represented by the past Coarse Tokens), this model is trained to estimate:
$$ p(y_t^q \mid z, y_{<t}^{<Q'}, y_t^{<q})  \quad \text{for} \ q \le Q'$$
where the current generated semantic token depends on **Semantic tokens** $z$ and the **acoustic tokens** $ y_{<t}^{<Q'} $ and $ y_t^{<q} $ that references respectively to **previous and current audio parts**. In the formula, $Q'$ denotes the **number of Soundstream quantizer** dedicated to Coarse tokens generation.


In [ ]:
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16

train_dataset, valid_dataset = random_split(coarseDataset, [TRAINING_PERCENTAGE, 1 - TRAINING_PERCENTAGE])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    log_every_n_steps=1,
    #devices=1 if torch.cuda.is_available() else None
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "checkpoints/coarse-checkpoint-10h-colab.ckpt"
d_model=256 #1024
num_layers=3 #12
num_heads=4 #16
dim_feedforward=1024 #4096
audioDuration=5 #10
vocab_size=1024


if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming old model...")
    model = CoarseTransformer.load_from_checkpoint(checkpoint_path, d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)
else:
    print("No checkpoint found. Starting from scratch...")
    model = CoarseTransformer(d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)

In [ ]:
model.train()
if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming training...")
    # Pass the checkpoint path to trainer.fit
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader, ckpt_path=checkpoint_path)
else:
    print("No checkpoint found. Starting from scratch...")
    # Start training from scratch
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

### Fine Transformer: rebuilding a detailed audio waveform

<center><img src="reportImg/fine.png"></center>

At last, we need to generate the **acoustic details** of the extended audio, since we need to recreate the **hierarchical structure** of acoustic features needed for the SoundStream model. This is easily done through a third and last transformer, that given the Coarse tokens and the previous Fine tokens, is trained to generate new Fine tokens. In formula:

$$ p(y_t^q \mid y_{<t}^{\leq Q'}, y_{\geq t}^{> Q'}, y_t^{<q}) \quad \text{for} \ q > Q' $$

where the new tokens depends on the **Coarse tokens** $y_{<t}^{\leq Q'}$ and the **Fine tokens** $y_{\geq t}^{> Q'}$ and $y_t^{<q}$, belonging respectively to **previous and current audio parts**.

In [ ]:
TRAINING_PERCENTAGE = 0.8
BATCH_SIZE = 16

train_dataset, valid_dataset = random_split(coarseDataset, [TRAINING_PERCENTAGE, 1 - TRAINING_PERCENTAGE])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainer = pl.Trainer(
    max_epochs=30,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    log_every_n_steps=1,
    #devices=1 if torch.cuda.is_available() else None
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint_path = "checkpoints/fine-checkpoint-10h-colab.ckpt"
d_model=256 #1024
num_layers=3 #12
num_heads=4 #16
dim_feedforward=1024 #4096
audioDuration=1.5 #3
vocab_size=1024


if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming old model...")
    model = FineTransformer.load_from_checkpoint(checkpoint_path, d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)
else:
    print("No checkpoint found. Starting from scratch...")
    model = FineTransformer(d_model=d_model, num_layers = num_layers, num_heads=num_heads, k = int(d_model/num_heads), dim_feedforward=dim_feedforward, audioDuration = audioDuration, vocab_size = vocab_size, myDevice = device)

In [ ]:
model.train()
if os.path.exists(checkpoint_path):
    print(f"Checkpoint found at {checkpoint_path}. Resuming training...")
    # Pass the checkpoint path to trainer.fit
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader, ckpt_path=checkpoint_path)
else:
    print("No checkpoint found. Starting from scratch...")
    # Start training from scratch
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

Once we have generated both Coarse acoustic tokens and Fine acoustic tokens, we can reshape them into a format that resembles the original data provided by the Residual Vector Quantizer component of SoundStream. Finally we pass the Acoustic tokens into the Decoder part of Soundstream, that will return the final audio waveform.

<center><img src="reportImg/full.png"></center>

## Inference and results

## References

- AudioLM paper: https://arxiv.org/abs/2209.03143
- AudioLM blog post: https://research.google/blog/audiolm-a-language-modeling-approach-to-audio-generation/
- Relative positional embeddings implementation: https://github.com/AliHaiderAhmad001/Self-Attention-with-Relative-Position-Representations